In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [143]:
# Read and label data

test = pd.read_csv('/kaggle/input/titanic/test.csv')
train = pd.read_csv('/kaggle/input/titanic/train.csv')


features = ['Pclass', 'Sex', 'Age', 'SibSp','Parch', 'Fare', 'Embarked']


X = train[features]
y = train['Survived']

In [151]:
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

# One hot encode categorical features

X = pd.get_dummies(X)

# Impute missing data using Bayesian Ridge

imp = IterativeImputer(estimator = BayesianRidge())

X_train, X_val, y_train, y_val = train_test_split(X, y)

imp.fit(X_train)
X_train_imp = imp.transform(X_train)



In [152]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score

# Random forest model with tuned hyperparameters

imp.fit(X_val)
X_val_imp = imp.transform(X_val)

forest_model = RandomForestClassifier(n_estimators = 128, max_depth = 10, max_features = 3)

forest_model.fit(X_train_imp, y_train)

forest_preds_val = forest_model.predict(X_val_imp)
forest_preds_train = forest_model.predict(X_train_imp)

print(f'Forest Model Training Score:', f1_score(y_train, forest_preds_train))
print(f'Forest Model Validation Score: ', f1_score(y_val, forest_preds_val))

Forest Model Training Score: 0.9290060851926977
Forest Model Validation Score:  0.7631578947368423


In [154]:
# Train on entire dataset for submission

# Impute test set
X_test = test[features]
X_test = pd.get_dummies(X_test)
imp.fit(X_test)
X_test = imp.transform(X_test)

# Compute and predict
imp.fit(X)
X = imp.transform(X)
forest_model.fit(X, y)
forest_predictions = forest_model.predict(X_test)

# Save and output to csv
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': forest_predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
